# Exploratory Data Analysis of VGChartz.com Video Game Sales Data

## Features to look into:
- Proximity of release date to holiday
- Day of week of release date
- price of console
- price of game
- whether developer is in top 5 or not
- location of developer
- number of years the game has been released
## Things to note/do
- make sure to remove data of games that were included with console ie. WII SPORTS

In [1]:
# if needed: pip install requests or conda install requests
import requests
from bs4 import BeautifulSoup
import re
import lxml.html as lh
import pandas as pd

In [111]:
# Ping one page of search results in VGChartz.com
first_page_url_200 = 'http://www.vgchartz.com/games/games.php?page=1&results=200&name=&console=&keyword=&publisher=&genre=&order=Sales&ownership=Both&boxart=Both&banner=Both&showdeleted=&region=All&goty_year=&developer=&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=1&showcriticscore=1&showuserscore=1&showshipped=1&alphasort=&showmultiplat=No'
last_page_url_200 = 'http://www.vgchartz.com/games/games.php?page=279&results=200&name=&console=&keyword=&publisher=&genre=&order=Sales&ownership=Both&boxart=Both&banner=Both&showdeleted=&region=All&goty_year=&developer=&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=1&showcriticscore=1&showuserscore=1&showshipped=1&alphasort=&showmultiplat=No'
first_page_url_5000 = 'http://www.vgchartz.com/games/games.php?page=1&results=5000&name=&console=&keyword=&publisher=&genre=&order=Sales&ownership=Both&boxart=Both&banner=Both&showdeleted=&region=All&goty_year=&developer=&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=1&showcriticscore=1&showuserscore=1&showshipped=1&alphasort=&showmultiplat=No'


url=first_page_url_5000
response = requests.get(url)

### Check the Status
response.status_code # status code = 200 => OK

200

In [112]:
#Store the contents of the website under doc
page=response.text
soup = BeautifulSoup(page, "lxml")
doc = lh.fromstring(response.content)

In [113]:
# Selects the table with all the data in it on HTML using xpath
tr_elements = doc.xpath('//*[@id="generalBody"]/table')[0]


In [114]:
# Check where the table begins, it appears to be on index 2
[len(T) for T in tr_elements[:15]]

[2, 1, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17]

In [115]:
# Verify we selected the table with the correct number of rows
len(tr_elements.xpath("./tr"))

5003

In [116]:
# Find the names of games from the links
names_list = list()
for row in tr_elements.xpath('.//tr'):
    for td in row.xpath('.//td'):
        if not td.find('a') is None:
            names_list.append(td.find('a').text.strip()) 

In [117]:
# Parse non-image and non-URL info from the data table to a pandas DataFrame
row_dict={}
df=pd.DataFrame()
row_list= list()
for counter,row in enumerate(tr_elements.xpath(".//tr")):
    if counter > 2:
        row_list=[td.text for td in row.xpath(".//td")]
        row_dict[counter] = row_list
# Test finding elements

df=pd.DataFrame.from_dict(row_dict).transpose()
df.columns = ['position','game','blank','console','publisher','developer','vgchart_score',\
             'critic_score','user_score','total_shipped','total_sales',\
              'na_sales','pal_sales','japan_sales','other_sales',\
              'release_date','last_update']

In [118]:
# Console tags are stored as images, so we find the image tag and record its 'alt' value as text
consoles = list()
for img in soup.find_all('img'):
    if 'images/consoles'in img['src']:
        console_tag = (img['src'][17:-6])
        consoles.append(img['alt'])


In [119]:
# Correct the console and game columns using scraped values
df=df.reset_index().drop(columns = ['index','blank'])
df['console'] = consoles
df['game'] = names_list

In [120]:
# Verify that correct data is in table
df.head()

,position,game,console,publisher,developer,vgchart_score,critic_score,user_score,total_shipped,total_sales,na_sales,pal_sales,japan_sales,other_sales,release_date,last_update
0,1,Wii Sports,Wii,Nintendo,Nintendo EAD,N/A,7.7,N/A,82.86m,N/A,N/A,N/A,N/A,N/A,19th Nov 06,N/A
1,2,Super Mario Bros.,NES,Nintendo,Nintendo EAD,N/A,10.0,8.2,40.24m,N/A,N/A,N/A,N/A,N/A,18th Oct 85,N/A
2,3,Mario Kart Wii,Wii,Nintendo,Nintendo EAD,8.7,8.2,9.1,37.14m,N/A,N/A,N/A,N/A,N/A,27th Apr 08,11th Apr 18
3,4,PlayerUnknown's Battlegrounds,PC,PUBG Corporation,PUBG Corporation,N/A,N/A,N/A,36.60m,N/A,N/A,N/A,N/A,N/A,21st Dec 17,13th Nov 18
4,5,Wii Sports Resort,Wii,Nintendo,Nintendo EAD,8.8,8.0,8.8,33.09m,N/A,N/A,N/A,N/A,N/A,26th Jul 09,N/A


In [121]:
df.shape

(5000, 16)

In [122]:
df.tail()

,position,game,console,publisher,developer,vgchart_score,critic_score,user_score,total_shipped,total_sales,na_sales,pal_sales,japan_sales,other_sales,release_date,last_update
4995,4996,Just Dance 4,PS3,Ubisoft,Ubisoft,N/A,N/A,N/A,N/A,0.45m,0.26m,0.11m,N/A,0.08m,09th Oct 12,31st Jan 18
4996,4997,Pirates of the Caribbean: At World's End,Wii,Disney Interactive Studios,Eurocom Entertainment Software,N/A,N/A,N/A,N/A,0.45m,0.39m,0.02m,0.01m,0.04m,22nd May 07,N/A
4997,4998,Brunswick Pro Bowling,X360,Crave Entertainment,FarSight Studios,N/A,N/A,N/A,N/A,0.45m,0.38m,0.04m,N/A,0.03m,07th Dec 10,N/A
4998,4999,Style Savvy: Styling Star,3DS,Nintendo,syn Sophia,N/A,8.5,N/A,N/A,0.45m,N/A,0.29m,0.14m,0.02m,25th Dec 17,05th Sep 18
4999,5000,Tekken Advance,GBA,Namco,Eighting / Raizing,N/A,8.3,N/A,N/A,0.45m,0.32m,0.12m,N/A,0.01m,28th Jan 02,N/A


In [126]:
# Write a function that takes in a VGChartz URL and gives us all the data in their video game database
def scrape_vgchartz_videogame_db_page(url):
    
    response = requests.get(url)

    ### Check the Status
    assert(response.status_code == 200)," Website not OK " # status code = 200 => OK
    
    #Store the contents of the website under doc
    page=response.text
    soup = BeautifulSoup(page, "lxml")
    doc = lh.fromstring(response.content)
    
    # Selects the table with all the data in it on HTML using xpath
    target_table_path = doc.xpath('//*[@id="generalBody"]/table')[0]

    # Find column values that won't be scraped correctly with .text option
    names_list = find_names_column(target_table_path)
    consoles = find_console_tags(soup)
    
    # Parse non-image and non-URL info from the data table to a pandas DataFrame
    row_dict={}
    df=pd.DataFrame()
    row_list= list()
    for counter,row in enumerate(target_table_path.xpath(".//tr")):
        if counter > 2: # To skip header rows
            row_list=[td.text for td in row.xpath(".//td")]
            row_dict[counter] = row_list

    df=pd.DataFrame.from_dict(row_dict).transpose()
    df.columns = ['position','game','blank','console','publisher','developer','vgchart_score',\
                 'critic_score','user_score','total_shipped','total_sales',\
                  'na_sales','pal_sales','japan_sales','other_sales',\
                  'release_date','last_update']
    
    # Correct the console and game columns using scraped values
    
    df=df.reset_index().drop(columns = ['index','blank'])
    df['console'] = consoles
    df['game'] = names_list
    return df

In [124]:
# Find the names of games from the links
def find_names_column(table_path):
    names_list = list()
    for row in table_path.xpath('.//tr'):
        for td in row.xpath('.//td'):
            if not td.find('a') is None:
                names_list.append(td.find('a').text.strip()) 
    return names_list

In [125]:
def find_console_tags(soup):
    # Console tags are stored as images, so we find the image tag and record its 'alt' value as text
    consoles = list()
    for img in soup.find_all('img'):
        if 'images/consoles'in img['src']:
            # Cut file path elements from string
            console_tag = (img['src'][17:-6])
            consoles.append(img['alt'])
    return consoles


In [135]:
# We can 'hack' the URL to display any number of results per page. I'll leave it as an argument.
def scrape_all_vg_chartz_videogame_db(results_per_page):
    df = pd.DataFrame()
    current_page = 1
    games_left = True
    while games_left:
        url = 'http://www.vgchartz.com/games/games.php?page=' + str(current_page) +\
        '&results=' + str(results_per_page) + '&name=&console=&keyword=&publisher=&genre=&order=Sales&ownership\
        =Both&boxart=Both&banner=Both&showdeleted=&region=All&goty_year=&developer=&direction\
        =DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&\
        showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=1&\
        showcriticscore=1&showuserscore=1&showshipped=1&alphasort=&showmultiplat=No'
        new_df = scrape_vgchartz_videogame_db_page(url)
        df = df.append(new_df)

        #REMOVE LATER, TEST CONDIITON
      #  if current_page > 3:
       #     games_left = False
        print('Scraped page: ',current_page)
        if new_df.shape[0] < results_per_page:
            games_left = False
        current_page +=1
    print('Scraping done!')
    print('Total rows parsed = ', df.shape[0])
    return df.reset_index().drop(columns = 'index')

In [134]:
df=scrape_all_vg_chartz_videogame_db(10000)

Scraped page:  1
Scraped page:  2
Scraped page:  3
Scraped page:  4
Scraped page:  5
Scraped page:  6
Scraping done!


In [138]:
df.to_pickle('./FullVGChartzDatabase.zip',compression = 'zip')

FullVGChartzDatabase.zip   exploratory_analysis.ipynb
README.md


total 2640
-rw-r--r--  1 jchow  staff   1.3M Apr 11 16:46 FullVGChartzDatabase.zip
-rw-r--r--  1 jchow  staff   130B Apr  9 15:51 README.md
-rw-r--r--  1 jchow  staff    32K Apr 11 16:46 exploratory_analysis.ipynb
